<a href="https://colab.research.google.com/github/SRISUDIV/ML-Hackathon-TechGig/blob/main/CG'23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy import sparse
import numpy as np

In [ ]:
# Load the training dataset
train_df = pd.read_csv("/content/drive/MyDrive/Competition /code_glaadiator'23/Doceree-HCP_Train.csv", encoding='ISO-8859-1')

# Load the testing dataset
test_df = pd.read_csv("/content/drive/MyDrive/Competition /code_glaadiator'23/Doceree-HCP_Test.csv", encoding='ISO-8859-1')

In [ ]:
train_df.head()

In [ ]:
# Encode categorical columns using label encoding
categorical_columns = ['DEVICETYPE', 'PLATFORMTYPE', 'CHANNELTYPE']
label_encoder = LabelEncoder()
for column in categorical_columns:
    train_df[column] = label_encoder.fit_transform(train_df[column])

# Preprocess text columns with CountVectorizer
text_columns = ['URL', 'KEYWORDS']
vectorizers = {}
text_features = []
for column in text_columns:
    vectorizer = CountVectorizer()
    vectorizers[column] = vectorizer
    text_feature = vectorizer.fit_transform(train_df[column])
    text_features.append(text_feature)

# Combine categorical features and text features
categorical_features = train_df[categorical_columns].values
combined_features = sparse.hstack([categorical_features] + text_features)

# Split the dataset into features (X) and target (y)
X = combined_features
y = train_df['IS_HCP']
y = y.map(lambda x: 1 if x == 1.0 else 0)
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)


In [ ]:
import joblib

joblib.dump(model, "xgboost_model.joblib")
joblib.dump(vectorizer, "vectorizer.joblib")
joblib.dump(label_encoder, 'label_encoder.joblib')


['label_encoder.joblib']

In [ ]:
# Validate the model
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Validation Results:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Validation Results:
Accuracy: 0.9827979638406179
Precision: 0.9937068303914045
Recall: 0.9485714285714286
F1 Score: 0.9706146926536732


In [ ]:
# Load the saved model
import joblib

loaded_model = joblib.load("/content/drive/MyDrive/Competition /code_glaadiator'23/xgboost_model.joblib")
loaded_vectorizer = joblib.load("/content/drive/MyDrive/Competition /code_glaadiator'23/vectorizer.joblib")
loaded_label_encoder=joblib.load("/content/drive/MyDrive/Competition /code_glaadiator'23/label_encoder.joblib")

In [ ]:
label_encoder.classes_

array(['Website'], dtype=object)

In [ ]:
# Check and replace unseen categories in the test dataset
for column in categorical_columns:
    test_df[column] = np.where(test_df[column].isin(label_encoder.classes_), test_df[column], 'Website')
# Preprocess text columns in the test dataset
test_text_features = []
for column in text_columns:
    test_text_feature = vectorizers[column].transform(test_df[column])
    test_text_features.append(test_text_feature)

# Convert categorical features to numeric
test_categorical_features = test_df[categorical_columns].apply(lambda x: label_encoder.transform(x))

# Convert categorical features to sparse matrix
test_categorical_features = sparse.csr_matrix(test_categorical_features.values)

# Combine categorical features and text features in the test dataset
test_combined_features = sparse.hstack([test_categorical_features] + test_text_features, format='csr')

# Make predictions on the test dataset
y_pred = model.predict(test_combined_features)

In [ ]:
# Make predictions on the test dataset
y_pred = loaded_model.predict(test_combined_features)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
final_train = pd.read_csv("/content/drive/MyDrive/Competition /code_glaadiator'23/Doceree-HCP_Train.csv", encoding='ISO-8859-1')
final_test = pd.read_csv("/content/drive/MyDrive/Competition /code_glaadiator'23/Doceree-HCP_Test.csv", encoding='ISO-8859-1')

In [ ]:
#Split the dataset into training and testing sets
X = final_train['KEYWORDS']
y = final_train['TAXONOMY'].fillna("")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Preprocess the keywords
def preprocess_keywords(keywords):
    # Remove unwanted characters, convert to lowercase, and split
    keywords = keywords.strip("[]'").lower().split('|')
    return ' '.join(keywords)

X_train = X_train.apply(preprocess_keywords)
X_test = X_test.apply(preprocess_keywords)

In [ ]:
X_train

54558     hepatology ophthalmology gastroenterology rheu...
81192     false sleep disorders general cardiology neuro...
17088     asthma psychiatry medicine clinical pediatrics...
67920     family practice drainage clinical dermatology ...
112199    arthritis urology oncology rheumatology clinic...
                                ...                        
76820     false general cardiology controlled laser radi...
110268    family practice drainage clinical dermatology ...
103694    small hospitals nephrology emergency medicine ...
860       cardiovascular clinical rheumatology transplan...
15795     health medicine chronic total cardiology contr...
Name: KEYWORDS, Length: 91149, dtype: object

In [ ]:
#Convert keywords into feature vectors using a vectorizer (e.g., TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)


In [ ]:
from sklearn.svm import SVC

# Train a SVM classifier
clf = SVC()
clf.fit(X_train_vectors, y_train)


SVC()

In [ ]:
# Step 5: Evaluate the model
y_pred = clf.predict(X_test_vectors)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8148148148148148


In [ ]:
joblib.dump(clf, "_model.joblib")

In [ ]:
# Step 6: Make predictions on new data
final = final_test['KEYWORDS'].values
new_keywords = [preprocess_keywords(keyword) for keyword in final]
new_keywords_vector = vectorizer.transform(new_keywords)
predicted_taxonomy = clf.predict(new_keywords_vector)
print("Predicted TAXONOMY:", predicted_taxonomy)



Predicted TAXONOMY: ['-' '-' '-' ... '-' '-' '-']


In [ ]:
final

array(['Family Practice|Drainage|Clinical|Dermatology|Ophthalmology|Medicine|Allergic|General|Diagnosis|Neurology|Hepatology|Health|Cardiovascular|Surgical|Psychiatry|Oral|Urology|Cardiology|Gastroenterology|Orthopedics|Erythema|Arthritis|Asthma|Cellulitis|Pediatrics|Surgery|False|Oncology|Rheumatology',
       'General|Clinical|Operative|Medicine|Cardiology|Oncology|Surgery|False|Urology|Laser|Surgical|Ophthalmology',
       'Abortion|Anxiety Disorders|Apnea|False|Trauma|Mental Health|Psychiatry|Urology|Cardiology|Medicine|Health|Bipolar and Related Disorders|General|Clinical|Myocardial Infarction|Conduct Disorders',
       ...,
       'small|hospitals|nephrology|emergency medicine|radiology|rheumatology|psychiatry|forensic|oncology|forensic medicine|specialties|rehabilitation|plastic surgery|trauma|educational|anesthesiology|pain management|gastroenterology|radiosurgery|acos|preventive|salivary gland|clinical|proton therapy|general practice|transplantation|urology|sports medicine|car

In [ ]:
# Output file for Scoring Solution
output_df = test_df[['ID']]  # Include the user platform ID in the output file
output_df['IS_HCP'] = y_pred

# Save the output file for the Scoring Solution
output_df.to_csv("scoring_solution.csv", index=False)



<ipython-input-43-18fed58b39f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['IS_HCP'] = y_pred


In [ ]:
sf=pd.read_csv("/content/scoring_solution.csv")

In [ ]:
sf.head()

,ID,IS_HCP
0,115501,0
1,115502,1
2,115503,0
3,115504,0
4,115505,1


In [ ]:
import pandas as pd

cf=pd.read_csv("/content/complete_solution.csv")

In [ ]:
cf.head()

,ID,TAXONOMY,IS_HCP
0,115501,NaN,0
1,115502,2084P0800X,1
2,115503,NaN,0
3,115504,NaN,0
4,115505,NaN,1
